In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from adapters import *
from dataset import NumpyCsvDataSet
from runner import ExptRunner
from networks.autoencoder import AutoEncoder
from networks.composedautoencoder import ComposedAutoEncoder
from networks.DenseAutoEncoder import DenseAutoEncoder
from networks.ConvVae import ConvVae
from networks.vae import VAE
from networks.etpr.autoencoder import autoencoder as etprautoenc
from networks.dense import Dense
from networks.dense import DenseForPolicy
from networks.lossfunctions import *
from helpers import ReportResults
import utils

In [3]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

In [4]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [5]:
start = time.time()
data_folder = 'data/demoplanner/version1'
v1_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v1_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 28.037065029144287 sec


In [19]:
start = time.time()
data_folder = 'data/demoplanner/version2'
v2_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v2_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 301.2297029495239 sec


In [6]:
train_data = v1_train_data
test_data = v1_test_data

In [7]:
print (train_data.data.shape)
print (test_data.data.shape)

print (train_data.data.device)
print (test_data.data.device)

print (train_data.data.dtype)
print (test_data.data.dtype)

torch.Size([13280, 2058])
torch.Size([1341, 2058])
cpu
cpu
torch.float32
torch.float32


In [8]:
expts = [
    ['DynamicsX-V1-Dense-MSE', lambda: Dense([4, 16, 1], last_act = 'tanh'), dynamics_input_adapter, mse_loss_adapter, dynamics_grad_gt_X_unscaled, None],
]

In [10]:
train_epochs = 5
runners = []
for i in range(len(expts)):
    inputs = expts[i]
    net = inputs[1]()
    net = net.to(device)
    runner = ExptRunner(inputs[0], net, train_data, test_data, 
                        inputs[2], inputs[3], inputs[4], data_to_img_func=inputs[5], device=device)
    print ("Log folder for expt {}: {}".format(i, runner.log_folder))
    runner.train(train_epochs, shouldShowReconstruction=False)
    runner.test()
    runners.append(runner)

Log folder for expt 0: runs/04-10-17-48-38-DynamicsX-V1-Dense-MSE


In [11]:
def build_reporter(checkpoint_file, useL1Loss=False):
    rr = ReportResults(test_data.data, train_data.data, device,
                       checkpoint_file, useL1Loss)
    return rr

In [18]:
for checkpoint_file in utils.enumerate_files():
    rr = build_reporter(checkpoint_file)
    print ('--------------------------------------------------------------')
    print (checkpoint_file)
    print ('--------------------------------------------------------------')
    rr.eval_test_samples()

In [19]:
print('Time,Goal,DataSet,Network,Loss(Training),TestLoss,TrainLoss')
for checkpoint_file in utils.enumerate_files():
    rr = build_reporter(checkpoint_file)
    rr.eval_test_samples()

Time,Goal,DataSet,Network,Loss(Training),TestLoss,TrainLoss


In [31]:
rr.generate_csv_comparison_report(test_data_float, train_data_float,
                   rootdir='runs/04-07')

Time,Goal,DataSet,Network,Loss(Training),TestLoss,TrainLoss
04:07:17:58:07,Policy,V1,Dense,CE,1.2633792161941528,1.2391564846038818
04:07:19:03:09,Policy,V2,Dense,CE,1.1975611448287964,1.1838151216506958
04:07:19:16:20,Policy,V2,Dense,CE,1.1645100116729736,1.177262306213379
04:08:05:39:29,Policy,V2,DenseForPolicy,CE,1.4296996593475342,1.3790206909179688
04:08:05:48:18,Policy,V2,DenseForPolicy,CE,1.3616671562194824,1.3848485946655273
04:08:10:17:06,Policy,V2,Dense,CE,2.801307346089743e-06,2.7218900413572555e-06
04:08:10:24:50,Policy,V2,Dense,CE,1.96190711676536e-07,1.5136279785110673e-07


In [32]:
sample = torch.cat((test_data.data[1055:1060], test_data.data[1230:1235]), axis=0)
ground_truth = policy_groud_truth_class_adapter(sample).to(device)
for folder, net in rr.build_net(rootdir='runs/04-07/'):
    print ('Folder: ' + folder)
    op_batch, loss = rr.run_mini_batch(net, sample)
    x_loss = F.cross_entropy(op_batch[:, :3], ground_truth[:, 0], reduction='none')
    y_loss = F.cross_entropy(op_batch[:, 3:], ground_truth[:, 1], reduction='none')
    print (ground_truth)
    print (op_batch)
    print (loss)
    print (x_loss)
    print (y_loss)
    print ('-----------------------------------------')

Folder: 04-07-17-58-07-Policy-V1-Dense-CE
tensor([[0, 1],
        [0, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [2, 0],
        [2, 0],
        [2, 0],
        [2, 0],
        [2, 1]])
tensor([[9.9892e-01, 1.1768e-02, 4.4407e-06, 6.9097e-01, 4.3719e-08, 5.8148e-05],
        [9.8408e-01, 4.1556e-02, 1.0096e-03, 9.8178e-01, 5.3358e-06, 1.9832e-04],
        [1.0000e+00, 1.0968e-05, 2.3005e-34, 5.3218e-30, 2.1840e-32, 2.9180e-05],
        [1.0000e+00, 4.4320e-05, 4.1472e-32, 7.9823e-29, 2.2644e-30, 4.3043e-04],
        [9.9999e-01, 1.9323e-04, 6.4159e-30, 1.2560e-27, 2.0004e-28, 5.3637e-03],
        [2.7807e-11, 4.0247e-02, 9.9016e-01, 9.9007e-01, 3.9920e-02, 2.1634e-12],
        [1.2382e-08, 3.5409e-02, 9.9082e-01, 9.9139e-01, 3.6082e-02, 8.6834e-10],
        [4.4673e-06, 3.4880e-02, 9.9111e-01, 9.9073e-01, 3.2762e-02, 3.6731e-07],
        [1.6327e-03, 3.4472e-02, 9.9060e-01, 9.8911e-01, 2.6565e-02, 1.5856e-04],
        [0.0000e+00, 7.8143e-03, 9.9971e-01, 0.0000e+00, 9.

In [19]:
rr = ReportResults(map_netname_net, device, 
                   policy_input_adapter, policy_groud_truth_adapter,
                   loss_adapter_func = l1_loss_adapter)

In [20]:
rr.generate_csv_comparison_report(test_data.data, train_data.data,
                   rootdir='runs/04-08')

Time,Goal,DataSet,Network,Loss(Training),TestLoss,TrainLoss
04:08:07:35:53,Policy,V2,Dense,MSE,0.0009246765985153615,0.0009788279421627522
04:08:07:29:36,Policy,V2,Dense,MSE,0.015583701431751251,0.016864705830812454


In [21]:
sample = torch.cat((test_data.data[1055:1060], test_data.data[1230:1235]), axis=0)
ground_truth = policy_groud_truth_adapter(sample).to(device)
for folder, net in rr.build_net(rootdir='runs/04-08/'):
    print ('Folder: ' + folder)
    op_batch, loss = rr.run_mini_batch(net, sample)
    print (ground_truth)
    print (op_batch)
    print (loss)

Folder: 04-08-07-35-53-Policy-V2-Dense-MSE
tensor([[-1.,  0.],
        [-1., -1.],
        [-1.,  0.],
        [-1.,  0.],
        [-1.,  0.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1.,  0.]])
tensor([[-1.0000e+00, -5.9831e-04],
        [-9.9837e-01, -9.9866e-01],
        [-1.0000e+00, -2.0846e-03],
        [-1.0000e+00, -2.0014e-03],
        [-1.0000e+00, -1.9175e-03],
        [ 9.9764e-01, -9.9867e-01],
        [ 9.9765e-01, -9.9868e-01],
        [ 9.9766e-01, -9.9868e-01],
        [ 9.9767e-01, -9.9868e-01],
        [ 1.0000e+00,  7.1299e-04]], grad_fn=<TanhBackward>)
tensor(0.0012, grad_fn=<MeanBackward0>)
Folder: 04-08-07-29-36-Policy-V2-Dense-MSE
tensor([[-1.,  0.],
        [-1., -1.],
        [-1.,  0.],
        [-1.,  0.],
        [-1.,  0.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1.,  0.]])
tensor([[-0.9994, -0.0311],
        [-0.9682, -0.9674],
        [-0.9995,  0.0176],
   

In [ ]:
np.set_printoptions(**opt)